In [1]:
import numpy as np 
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from sklearn.model_selection import train_test_split

In [2]:

data = pd.read_csv("/users/mt/Documents/TCEproject/data/data3.csv")
data = data.drop(['Unnamed: 0'], axis =1)
data 

,CD4NadirBeforeTCE,BaselineCD4Count,pastrawLoad,pastCD4count,BaselineRNAlog
0,5,100,1000,40,0
1,267,296,15849,5,1
2,296,369,3981,164,0
3,19,19,19953,156,1
4,106,207,15849,472,0
...,...,...,...,...,...
109147,50,101,501,51,0
109148,213,403,79,112,1
109149,224,247,3981,244,1
109150,9,81,5012,88,1


In [3]:
y = data['BaselineRNAlog'].values
X = data.drop(['BaselineRNAlog'],1).values

In [4]:
X

array([[    5,   100,  1000,    40],
       [  267,   296, 15849,     5],
       [  296,   369,  3981,   164],
       ...,
       [  224,   247,  3981,   244],
       [    9,    81,  5012,    88],
       [  151,   207,  1995,   416]])

In [5]:
y

array([0, 1, 0, ..., 1, 1, 0])

In [6]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [7]:
scaler = StandardScaler()
scaler.fit(X)

StandardScaler()

In [8]:
X = (scaler.transform(X))
X

array([[-1.0433021 , -0.96295819, -0.26904034, -1.34010951],
       [ 0.97377943,  0.01935292, -0.12737008, -1.4959953 ],
       [ 1.19704418,  0.38521369, -0.24059943, -0.78782844],
       ...,
       [ 0.6427317 , -0.22622486, -0.24059943, -0.43151807],
       [-1.01250696, -1.05818223, -0.23076294, -1.12632329],
       [ 0.08072043, -0.42669651, -0.25954732,  0.33454923]])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state = 0)
# X_test, X_val, y_test, y_val     = train_test_split(X_test,y_test, test_size=0.5, random_state=42)

In [10]:
# # for i in [X_train,X_test, X_val]:
#     print(round(len(i) / len(y), 2))

In [11]:
BATCH_SIZE = 64

torch_X_train = torch.from_numpy(X_train).type(torch.LongTensor)
torch_y_train = torch.from_numpy(y_train).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set.
torch_X_test = torch.from_numpy(X_test).type(torch.LongTensor)
torch_y_test = torch.from_numpy(y_test).type(torch.LongTensor) # data type is long

# torch_X_val = torch.from_numpy(X_val).type(torch.LongTensor)
# torch_y_val = torch.from_numpy(y_val).type(torch.LongTensor) # data type is long


# Pytorch train and test sets
train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)
# val  = torch.utils.data.TensorDataset(torch_X_val, torch_y_val)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader  = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)
# val_loader   = torch.utils.data.DataLoader(val, batch_size  = BATCH_SIZE, shuffle = False)

In [12]:

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

In [13]:
for data,target in train_loader:break
target

tensor([0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0])

In [14]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,2)
        self.dropout = nn.Dropout(p =0.2)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x))
        return x


In [15]:
model = Model()
model.parameters

<bound method Module.parameters of Model(
  (fc1): Linear(in_features=4, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)>

In [16]:
for p in model.parameters():
    print(p.shape)

torch.Size([64, 4])
torch.Size([64])
torch.Size([32, 64])
torch.Size([32])
torch.Size([2, 32])
torch.Size([2])


In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [23]:
def fit(model, train_loader):
    model.train()
    epochs  = 100
    for epoch in range(epochs):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            batch_idx = batch_idx +1
            X_batch = X_batch.float()           
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
            # Total correct predictions
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == y_batch).sum()
            if batch_idx % 10 == 1:
                print('Epoch : {},\t Loss {} \t Accuracy:{:.3f}%'.format(epoch, loss.item(), correct*100/float(BATCH_SIZE*(batch_idx+1))))
        

In [24]:
fit(model, train_loader)

<ipython-input-14-bd21634930c5>:11: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(self.fc3(x))


Epoch : 0,	 Loss 0.6727883219718933 	 Accuracy:27.344%
Epoch : 0,	 Loss 0.6092690825462341 	 Accuracy:61.068%
Epoch : 0,	 Loss 0.5284201502799988 	 Accuracy:64.560%
Epoch : 0,	 Loss 0.6242910027503967 	 Accuracy:65.674%
Epoch : 0,	 Loss 0.6279482245445251 	 Accuracy:65.327%
Epoch : 0,	 Loss 0.6400503516197205 	 Accuracy:65.775%
Epoch : 0,	 Loss 0.5591410994529724 	 Accuracy:65.902%
Epoch : 0,	 Loss 1.1001302003860474 	 Accuracy:65.799%
Epoch : 0,	 Loss 0.5675391554832458 	 Accuracy:66.139%
Epoch : 0,	 Loss 0.5657838582992554 	 Accuracy:66.661%
Epoch : 0,	 Loss 0.5329204201698303 	 Accuracy:66.682%
Epoch : 0,	 Loss 0.7385886907577515 	 Accuracy:66.616%
Epoch : 0,	 Loss 0.5341941118240356 	 Accuracy:66.829%
Epoch : 0,	 Loss 0.5439589023590088 	 Accuracy:66.844%
Epoch : 0,	 Loss 0.6011593341827393 	 Accuracy:66.714%
Epoch : 0,	 Loss 0.5915491580963135 	 Accuracy:66.643%
Epoch : 0,	 Loss 0.6240463256835938 	 Accuracy:66.705%
Epoch : 0,	 Loss 0.5326815247535706 	 Accuracy:66.888%
Epoch : 0,

Epoch : 1,	 Loss 0.7470093965530396 	 Accuracy:65.538%
Epoch : 1,	 Loss 0.5816639065742493 	 Accuracy:65.911%
Epoch : 1,	 Loss 0.5659754872322083 	 Accuracy:66.457%
Epoch : 1,	 Loss 0.5649219155311584 	 Accuracy:66.437%
Epoch : 1,	 Loss 0.7369814515113831 	 Accuracy:66.420%
Epoch : 1,	 Loss 0.5411856174468994 	 Accuracy:66.624%
Epoch : 1,	 Loss 0.5465442538261414 	 Accuracy:66.655%
Epoch : 1,	 Loss 0.6089301705360413 	 Accuracy:66.582%
Epoch : 1,	 Loss 0.5802648067474365 	 Accuracy:66.519%
Epoch : 1,	 Loss 0.608396053314209 	 Accuracy:66.676%
Epoch : 1,	 Loss 0.5340256690979004 	 Accuracy:66.870%
Epoch : 1,	 Loss 0.6687355041503906 	 Accuracy:66.861%
Epoch : 1,	 Loss 0.6000755429267883 	 Accuracy:66.935%
Epoch : 1,	 Loss 0.6050867438316345 	 Accuracy:66.778%
Epoch : 1,	 Loss 0.5903275609016418 	 Accuracy:66.856%
Epoch : 1,	 Loss 0.5340818166732788 	 Accuracy:66.913%
Epoch : 1,	 Loss 0.5916910767555237 	 Accuracy:66.999%
Epoch : 1,	 Loss 0.5822316408157349 	 Accuracy:66.852%
Epoch : 1,	

Epoch : 2,	 Loss 0.582783043384552 	 Accuracy:66.334%
Epoch : 2,	 Loss 0.614157497882843 	 Accuracy:66.454%
Epoch : 2,	 Loss 0.5366976261138916 	 Accuracy:66.615%
Epoch : 2,	 Loss 0.6614063382148743 	 Accuracy:66.621%
Epoch : 2,	 Loss 0.5905039310455322 	 Accuracy:66.724%
Epoch : 2,	 Loss 0.6638593077659607 	 Accuracy:66.553%
Epoch : 2,	 Loss 0.598408043384552 	 Accuracy:66.627%
Epoch : 2,	 Loss 0.522227942943573 	 Accuracy:66.681%
Epoch : 2,	 Loss 0.58896404504776 	 Accuracy:66.790%
Epoch : 2,	 Loss 0.5875501036643982 	 Accuracy:66.645%
Epoch : 2,	 Loss 0.6186258792877197 	 Accuracy:66.580%
Epoch : 2,	 Loss 0.6190006732940674 	 Accuracy:66.651%
Epoch : 2,	 Loss 0.5752046704292297 	 Accuracy:66.768%
Epoch : 2,	 Loss 0.5235167145729065 	 Accuracy:66.805%
Epoch : 2,	 Loss 0.5918769240379333 	 Accuracy:66.679%
Epoch : 2,	 Loss 0.7063215970993042 	 Accuracy:66.680%
Epoch : 2,	 Loss 0.5653603672981262 	 Accuracy:66.787%
Epoch : 2,	 Loss 0.6172630190849304 	 Accuracy:66.872%
Epoch : 2,	 Loss

Epoch : 3,	 Loss 0.6089986562728882 	 Accuracy:66.019%
Epoch : 3,	 Loss 0.6101817488670349 	 Accuracy:66.097%
Epoch : 3,	 Loss 0.5227232575416565 	 Accuracy:66.174%
Epoch : 3,	 Loss 0.5897935628890991 	 Accuracy:66.298%
Epoch : 3,	 Loss 0.5916061997413635 	 Accuracy:66.161%
Epoch : 3,	 Loss 0.6195939183235168 	 Accuracy:66.065%
Epoch : 3,	 Loss 0.6082109212875366 	 Accuracy:66.114%
Epoch : 3,	 Loss 0.567493200302124 	 Accuracy:66.211%
Epoch : 3,	 Loss 0.5509610772132874 	 Accuracy:66.284%
Epoch : 3,	 Loss 0.5983982682228088 	 Accuracy:66.176%
Epoch : 3,	 Loss 0.7028771042823792 	 Accuracy:66.246%
Epoch : 3,	 Loss 0.5512976050376892 	 Accuracy:66.341%
Epoch : 3,	 Loss 0.6254981756210327 	 Accuracy:66.426%
Epoch : 3,	 Loss 0.6125121116638184 	 Accuracy:66.416%
Epoch : 3,	 Loss 0.566438615322113 	 Accuracy:66.511%
Epoch : 3,	 Loss 0.5824028253555298 	 Accuracy:66.420%
Epoch : 3,	 Loss 0.6318325400352478 	 Accuracy:66.406%
Epoch : 3,	 Loss 0.6225932836532593 	 Accuracy:66.381%
Epoch : 3,	 

...

Epoch : 95,	 Loss 0.5798620581626892 	 Accuracy:66.945%
Epoch : 95,	 Loss 0.6145990490913391 	 Accuracy:66.959%
Epoch : 95,	 Loss 0.565168559551239 	 Accuracy:66.982%
Epoch : 95,	 Loss 0.6032955646514893 	 Accuracy:66.976%
Epoch : 95,	 Loss 0.6202006936073303 	 Accuracy:66.958%
Epoch : 95,	 Loss 0.6886117458343506 	 Accuracy:66.943%
Epoch : 95,	 Loss 0.5581325888633728 	 Accuracy:66.986%
Epoch : 95,	 Loss 0.5720822811126709 	 Accuracy:66.985%
Epoch : 95,	 Loss 0.586916983127594 	 Accuracy:66.967%
Epoch : 95,	 Loss 0.5854211449623108 	 Accuracy:66.969%
Epoch : 95,	 Loss 0.6401501893997192 	 Accuracy:66.958%
Epoch : 95,	 Loss 0.5839869976043701 	 Accuracy:66.969%
Epoch : 95,	 Loss 0.5638279914855957 	 Accuracy:66.993%
Epoch : 95,	 Loss 0.5603168606758118 	 Accuracy:67.012%
Epoch : 95,	 Loss 0.5718371868133545 	 Accuracy:67.022%
Epoch : 95,	 Loss 0.6495798230171204 	 Accuracy:66.979%
Epoch : 95,	 Loss 0.615410327911377 	 Accuracy:66.986%
Epoch : 95,	 Loss 0.583165168762207 	 Accuracy:66.9

Epoch : 96,	 Loss 0.5835174918174744 	 Accuracy:66.998%
Epoch : 96,	 Loss 0.5591689944267273 	 Accuracy:67.021%
Epoch : 96,	 Loss 0.5619140863418579 	 Accuracy:67.038%
Epoch : 96,	 Loss 0.5708699226379395 	 Accuracy:67.047%
Epoch : 96,	 Loss 0.6487422585487366 	 Accuracy:67.007%
Epoch : 96,	 Loss 0.6138763427734375 	 Accuracy:67.013%
Epoch : 96,	 Loss 0.582273542881012 	 Accuracy:67.013%
Epoch : 96,	 Loss 0.5677030086517334 	 Accuracy:67.021%
Epoch : 96,	 Loss 0.5578616857528687 	 Accuracy:67.015%
Epoch : 96,	 Loss 0.6667364835739136 	 Accuracy:67.035%
Epoch : 96,	 Loss 0.6034591794013977 	 Accuracy:67.025%
Epoch : 96,	 Loss 0.6034762263298035 	 Accuracy:67.031%
Epoch : 96,	 Loss 0.5475638508796692 	 Accuracy:67.045%
Epoch : 96,	 Loss 0.5721710324287415 	 Accuracy:67.050%
Epoch : 96,	 Loss 0.5217925906181335 	 Accuracy:67.054%
Epoch : 96,	 Loss 0.6408941149711609 	 Accuracy:67.040%
Epoch : 96,	 Loss 0.6205941438674927 	 Accuracy:67.045%
Epoch : 96,	 Loss 0.5436878204345703 	 Accuracy:6

Epoch : 97,	 Loss 0.5617761015892029 	 Accuracy:66.806%
Epoch : 97,	 Loss 0.5661166310310364 	 Accuracy:66.798%
Epoch : 97,	 Loss 0.6766394376754761 	 Accuracy:66.820%
Epoch : 97,	 Loss 0.5907659530639648 	 Accuracy:66.803%
Epoch : 97,	 Loss 0.6086851358413696 	 Accuracy:66.812%
Epoch : 97,	 Loss 0.5527915358543396 	 Accuracy:66.824%
Epoch : 97,	 Loss 0.5829157829284668 	 Accuracy:66.822%
Epoch : 97,	 Loss 0.5104877948760986 	 Accuracy:66.831%
Epoch : 97,	 Loss 0.6552772521972656 	 Accuracy:66.812%
Epoch : 97,	 Loss 0.6284598112106323 	 Accuracy:66.823%
Epoch : 97,	 Loss 0.560120165348053 	 Accuracy:66.812%
Epoch : 97,	 Loss 0.639787495136261 	 Accuracy:66.822%
Epoch : 97,	 Loss 0.6016002893447876 	 Accuracy:66.818%
Epoch : 97,	 Loss 0.554638147354126 	 Accuracy:66.820%
Epoch : 97,	 Loss 0.5135049819946289 	 Accuracy:66.843%
Epoch : 97,	 Loss 0.6490347981452942 	 Accuracy:66.832%
Epoch : 97,	 Loss 0.5823516845703125 	 Accuracy:66.831%
Epoch : 97,	 Loss 0.6658499240875244 	 Accuracy:66.

Epoch : 98,	 Loss 0.5715103149414062 	 Accuracy:66.886%
Epoch : 98,	 Loss 0.5116217136383057 	 Accuracy:66.889%
Epoch : 98,	 Loss 0.6542620062828064 	 Accuracy:66.875%
Epoch : 98,	 Loss 0.6230624914169312 	 Accuracy:66.883%
Epoch : 98,	 Loss 0.5465381145477295 	 Accuracy:66.877%
Epoch : 98,	 Loss 0.6408481001853943 	 Accuracy:66.890%
Epoch : 98,	 Loss 0.5994428396224976 	 Accuracy:66.891%
Epoch : 98,	 Loss 0.5541113615036011 	 Accuracy:66.892%
Epoch : 98,	 Loss 0.5130360722541809 	 Accuracy:66.914%
Epoch : 98,	 Loss 0.6530041694641113 	 Accuracy:66.902%
Epoch : 98,	 Loss 0.5812711715698242 	 Accuracy:66.904%
Epoch : 98,	 Loss 0.662815272808075 	 Accuracy:66.914%
Epoch : 98,	 Loss 0.6215682029724121 	 Accuracy:66.892%
Epoch : 98,	 Loss 0.6281911730766296 	 Accuracy:66.883%
Epoch : 98,	 Loss 0.5960023999214172 	 Accuracy:66.900%
Epoch : 98,	 Loss 0.585500955581665 	 Accuracy:66.904%
Epoch : 98,	 Loss 0.6137568950653076 	 Accuracy:66.885%
Epoch : 99,	 Loss 0.6789999604225159 	 Accuracy:26

Epoch : 99,	 Loss 0.6516016125679016 	 Accuracy:66.877%
Epoch : 99,	 Loss 0.5843684673309326 	 Accuracy:66.879%
Epoch : 99,	 Loss 0.659727931022644 	 Accuracy:66.888%
Epoch : 99,	 Loss 0.62326979637146 	 Accuracy:66.867%
Epoch : 99,	 Loss 0.6296614408493042 	 Accuracy:66.863%
Epoch : 99,	 Loss 0.5983996391296387 	 Accuracy:66.879%
Epoch : 99,	 Loss 0.5924882888793945 	 Accuracy:66.881%
Epoch : 99,	 Loss 0.6147127151489258 	 Accuracy:66.861%


In [20]:
def evaluate(model):
    correct = 0 
    for test_data, test_labels in test_loader:
        test_data = test_data.float()
        output = model(test_data)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f}% ".format( float(correct) / (len(test_loader)*BATCH_SIZE)))



In [21]:
evaluate(model)

<ipython-input-14-bd21634930c5>:11: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(self.fc3(x))


Test accuracy:0.678% 
